In [1]:
from SAC import Actor, Critic, ReplayBuffer, Value
import sys
sys.path.append('C:\\Users\\Максимилиан\\Desktop\\Skoltech\\Reinforcement learning\\Final project')
from Dynamics import get_next_state, state_to_coords, get_energy,normalize_angle
from Environment import DoublePendulumEnv, ObservationSpaceCartPole
from config_SAC import config
from Agent import Agent
from utils import plot_learning_curve

In [2]:
from torch import nn
import torch
from torch.utils.tensorboard import SummaryWriter
import gym
from torch.distributions import Normal
from gym.spaces import Box
import random
from IPython.display import clear_output
from torch.distributions import MultivariateNormal
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
state0 = np.array([0.0001,np.pi/2,np.pi/2,
                0.0001,0.0001,0])
state = state0
max_initial_angle = 0
dt = 0.02

In [4]:
env = DoublePendulumEnv(init_state=state, dt=config['dt'], max_initial_angle = 0)

Environment initialized


C:\Users\Максимилиан\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [5]:
dir_path = os.path.join('SAC', config['env_name'])
if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [6]:
buffer = ReplayBuffer(config['memory_size'], env.observation_space.shape, env.action_space.shape)

In [7]:
 agent = Agent(gamma= config['gamma'], alpha=config['alpha'], beta=config['beta'], state_dims=env.observation_space.shape,
                  action_dims=env.action_space.shape, max_action=env.action_space.high[0],
                  fc1_dim=config['fc1_dim'], fc2_dim=config['fc2_dim'], memory_size=config['memory_size'],
                  batch_size=config['batch_size'], tau=config['tau'], update_period=config['update_period'],
                  reward_scale=config['reward_scale'], warmup=config['warmup'], reparam_noise_lim=config['reparam_noise_lim'],
                  name='SAC_'+config['env_name'], ckpt_dir=dir_path)

In [8]:
scores, avg_scores = [], []
best_score = -np.inf

for game in range(config['n_games']):
        observation, done = env.reset()

        score = 0
        counter = 0
        
        for i in range(0,301):
            
            if config['play']:
                action = agent.choose_action(observation, deterministic=True, reparameterize=False)
            else:
                action = agent.choose_action(observation, deterministic=False, reparameterize=False)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.store_transition(observation, action, reward, observation_, done)
            if not config['play']:
                agent.learn()
            observation = observation_
            counter += 1
            if done:
                break
        scores.append(score)
        avg_score = np.mean(scores[-100:])
        avg_scores.append(avg_score)

        print(f'| Game: {game:6.0f} | Score: {score:10.2f} | Best score: {best_score:10.2f} | '
              f'Avg score {avg_score:10.2f} | Learning inter: {agent.learn_iter:14.0f} |')

        if avg_score > best_score:
            best_score = avg_score
            if not config['play']:
                agent.save_model()
env.close()

if not config['play']:
    plot_learning_curve(scores, agent.full_path)

C:\Users\Максимилиан\Desktop\Skoltech\Reinforcement learning\Final project\SAC\Agent.py:72: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  state = T.tensor([state], dtype=T.float).to(self.actor.device)


| Game:      0 | Score:  111573.21 | Best score:       -inf | Avg score  111573.21 | Learning inter:             31 |
... saving checkpoint ...
... saving checkpoint ...
... saving checkpoint ...
... saving checkpoint ...
... saving checkpoint ...
| Game:      1 | Score:   82906.85 | Best score:  111573.21 | Avg score   97240.03 | Learning inter:             59 |
| Game:      2 | Score:   92602.28 | Best score:  111573.21 | Avg score   95694.11 | Learning inter:             87 |
| Game:      3 | Score:   87698.92 | Best score:  111573.21 | Avg score   93695.32 | Learning inter:            115 |
| Game:      4 | Score:   86596.23 | Best score:  111573.21 | Avg score   92275.50 | Learning inter:            143 |
| Game:      5 | Score:   88993.25 | Best score:  111573.21 | Avg score   91728.46 | Learning inter:            171 |
| Game:      6 | Score:   95298.82 | Best score:  111573.21 | Avg score   92238.51 | Learning inter:            200 |
| Game:      7 | Score:   95559.53 | Best sc

ValueError: Expected parameter loc (Tensor of shape (1, 1)) of distribution Normal(loc: tensor([[nan]], grad_fn=<AddmmBackward0>), scale: tensor([[nan]], grad_fn=<ClampBackward1>)) to satisfy the constraint Real(), but found invalid values:
tensor([[nan]], grad_fn=<AddmmBackward0>)